In [1]:
import torch
import pickle

with open( "x_train_hist.p", "rb" ) as file:
    x_train_hist = pickle.load(file)
    
with open( "y_train_hist.p", "rb" ) as file:
    y_train_hist = pickle.load(file)
    
with open( "x_valid_hist.p", "rb" ) as file:
    x_valid_hist = pickle.load(file)
    
with open( "y_valid_hist.p", "rb" ) as file:
    y_valid_hist = pickle.load(file)

In [2]:
import torch
import random
import numpy as np
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss
from tqdm.auto import tqdm

from mriqa_dataset import MRIQADataset
from networks import ClassicCNN, PhilsClassicCnn, CatNet

If you use TorchIO for your research, please cite the following paper:
Pérez-García et al., TorchIO: a Python library for efficient loading,
preprocessing, augmentation and patch-based sampling of medical images
in deep learning. Link: https://arxiv.org/abs/2003.04696



In [3]:
import torch_optimizer as optim
num_epochs = len(x_train_hist)

net = CatNet(num_classes=5)
net = net.cuda()

#optimizer = optim.Adam(net.parameters())
optimizer = optim.Ranger(
    net.parameters(),
    lr=1e-3,
    alpha=0.5,
    k=6,
    N_sma_threshhold=5,
    betas=(.95, 0.999),
    eps=1e-5,
    weight_decay=0
)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs*13)
ce = CrossEntropyLoss().cuda()

best_val_loss = 999

loss_csv = open('losses.csv', 'w')
loss_csv.write('epoch,training,validation\n')
num_mini_batches = 13
    
print("start training")
for epoch in range(num_epochs):
    epoch_loss = 0.0
    net.train()
    
    train_batches = x_train_hist[epoch]
    train_labels = y_train_hist[epoch]

    # train loop
    for sample, label in tqdm(zip(train_batches, train_labels), total=len(train_batches), leave=False):       
        sample = sample.cuda()
        label = label.cuda()

        prediction = net(sample)
        loss = ce(prediction, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(net.parameters(), 0.75)
        optimizer.step()

        epoch_loss += loss.item()

    print('[{}] train-loss: {}'.format(epoch, epoch_loss / num_mini_batches))
    loss_csv.write(str(epoch) + ',' + str(epoch_loss / num_mini_batches))
    loss_csv.flush()

    # validation loop
    net.eval()
    mean_validation_loss = 0
    num_validation_mini_batches = 11
    
    if epoch % 5 == 0:
        valid_batches = x_valid_hist[epoch//5]
        valid_labels = y_valid_hist[epoch//5]
        with torch.no_grad():
            for sample, label in tqdm(zip(valid_batches, valid_labels), total=len(valid_batches), leave=False):              
                sample = sample.cuda()
                label = label.cuda()

                prediction = net(sample)
                validation_loss = ce(prediction, label)

                mean_validation_loss += validation_loss.item()
            print('[{}] validation-loss: {}'.format(epoch, mean_validation_loss / num_validation_mini_batches))
            loss_csv.write(',' + str(mean_validation_loss / num_validation_mini_batches) + '\n')
            loss_csv.flush()

        # save best model
        if mean_validation_loss <= best_val_loss:
            torch.save({'epoch': epoch,
                        'model_state_dict': net.state_dict(),
                        'optimizer_state_dict': optimizer.state_dict(),
                        'loss': validation_loss.item()}, 'checkpoint_best')
            best_val_loss = mean_validation_loss

print('DONE.')

start training


..\torch\csrc\utils\python_arg_parser.cpp:756: UserWarning: This overload of addcmul_ is deprecated:
	addcmul_(Number value, Tensor tensor1, Tensor tensor2)
Consider using one of the following signatures instead:
	addcmul_(Tensor tensor1, Tensor tensor2, *, Number value)


[0] train-loss: 1.5663099105541523


[0] validation-loss: 1.6248558868061413


[1] train-loss: 1.4350514870423536


...

[220] validation-loss: 0.24854535626416857


[221] train-loss: 0.3396165370941162


[222] train-loss: 0.24585402126495653


[223] train-loss: 0.19631077005312994


[224] train-loss: 0.24056900349947122
DONE.


In [4]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [5]:
all_labels = []
all_predictions = []

with torch.no_grad():
    for batch, labels in tqdm(zip(x_train_hist, y_train_hist)):
        for minibatch, label in zip(batch, labels):
            sample = minibatch.cuda()

            prediction = net(sample)
            all_predictions.append(prediction.cpu())
            
            all_labels.append(label)

In [6]:
all_predictions = [item.argmax().tolist() for sublist in all_predictions for item in sublist]
all_labels = [item.tolist() for sublist in all_labels for item in sublist]
print(classification_report(all_labels, all_predictions))

              precision    recall  f1-score   support

           0       0.97      0.92      0.94      4592
           1       0.99      0.94      0.97      4372
           2       0.94      0.98      0.96      4542
           3       1.00      1.00      1.00      4555
           4       0.93      0.99      0.96      4439

    accuracy                           0.97     22500
   macro avg       0.97      0.97      0.97     22500
weighted avg       0.97      0.97      0.97     22500



In [ ]:
confusion_matrix(all_labels, all_predictions)

In [7]:
all_labels = []
all_predictions = []

with torch.no_grad():
    for batch, labels in tqdm(zip(x_valid_hist, y_valid_hist)):
        for minibatch, label in zip(batch, labels):
            sample = minibatch.cuda()

            prediction = net(sample)
            all_predictions.append(prediction.cpu())
            
            all_labels.append(label)

In [8]:
all_predictions = [item.argmax().tolist() for sublist in all_predictions for item in sublist]
all_labels = [item.tolist() for sublist in all_labels for item in sublist]
print(classification_report(all_labels, all_predictions))

              precision    recall  f1-score   support

           0       0.97      0.90      0.93       775
           1       0.98      0.94      0.96       862
           2       0.93      0.97      0.95       881
           3       1.00      1.00      1.00       855
           4       0.92      0.99      0.96       877

    accuracy                           0.96      4250
   macro avg       0.96      0.96      0.96      4250
weighted avg       0.96      0.96      0.96      4250



In [18]:
print(confusion_matrix(all_labels, all_predictions))

[[694   8  52   0  21]
 [ 19 808  11   0  24]
 [  1   0 854   0  26]
 [  0   0   0 855   0]
 [  0   6   4   0 867]]


In [10]:
torch.save({'epoch': epoch,
            'model_state_dict': net.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': validation_loss.item()}, 'final')

In [26]:
model = torch.load("checkpoint_best")

In [33]:
model = CatNet(num_classes=5)
optimizer = optim.Ranger(
    net.parameters(),
    lr=1e-3,
    alpha=0.5,
    k=6,
    N_sma_threshhold=5,
    betas=(.95, 0.999),
    eps=1e-5,
    weight_decay=0
)

checkpoint = torch.load("checkpoint_best")
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

model.eval()
model.cuda()

CatNet(
  (block1): Sequential(
    (0): Conv2d(1, 64, kernel_size=(7, 7), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): mish_layer()
  )
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (block2): skip_connection_block(
    (conv_il): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2))
    (batn_il): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv_ol): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (batn_ol): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv_sl): Conv2d(64, 64, kernel_size=(1, 1), stride=(2, 2))
    (batn_sl): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (block3): skip_connection_block(
    (conv_il): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2))
    (batn_il): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, tr

In [37]:
all_labels = []
all_predictions = []

with torch.no_grad():
    for batch, labels in tqdm(zip(x_valid_hist, y_valid_hist)):
        for minibatch, label in zip(batch, labels):
            sample = minibatch.cuda()

            prediction = model(sample)
            all_predictions.append(prediction.cpu())
            
            all_labels.append(label)

In [38]:
all_predictions = [item.argmax().tolist() for sublist in all_predictions for item in sublist]
all_labels = [item.tolist() for sublist in all_labels for item in sublist]
print(classification_report(all_labels, all_predictions))

              precision    recall  f1-score   support

           0       0.97      0.90      0.94       775
           1       0.92      0.94      0.93       862
           2       0.99      0.90      0.95       881
           3       1.00      0.99      1.00       855
           4       0.87      0.98      0.92       877

    accuracy                           0.95      4250
   macro avg       0.95      0.94      0.95      4250
weighted avg       0.95      0.95      0.95      4250

